This notebook is part of https://github.com/AudioSceneDescriptionFormat/splines, see also https://splines.readthedocs.io/.

[back to overview](finite-difference.ipynb)

# Non-Uniform Finite Difference Splines

Given the vertices $\boldsymbol{x}_i$
(and some [end conditions](end-conditions.ipynb)),
a *finite difference* spline can be constructed
using a [non-uniform Hermite spline](hermite-non-uniform.ipynb)
and setting its tangent vectors to

\begin{equation*}
\boldsymbol{\dot{x}}_i = \frac{1}{2} \left(
\frac{\boldsymbol{x}_i - \boldsymbol{x}_{i-1}}{t_i - t_{i-1}} +
\frac{\boldsymbol{x}_{i + 1} - \boldsymbol{x}_i}{t_{i + 1} - t_i}
\right).
\end{equation*}

This is really all that's needed,
but if you are interested in the basis matrix,
here's its derivation:

In [ ]:
import sympy as sp
sp.init_printing()

In [ ]:
from utility import NamedExpression, NamedMatrix

In [ ]:
t = sp.symbols('t')

We are considering the fifth spline segment
from $\boldsymbol{x}_4$ to $\boldsymbol{x}_5$,
but we will also need
the preceding value $\boldsymbol{x}_3$ and
the following value $\boldsymbol{x}_6$,
as well as all associated parameter values.

In [ ]:
x3, x4, x5, x6 = sp.symbols('xbm3:7')
t3, t4, t5, t6 = sp.symbols('t3:7')

In [ ]:
control_values_FD = sp.Matrix([x3, x4, x5, x6])
control_values_FD

We use the aforementioned expressions for the tangents at
$\boldsymbol{x}_4$ and $\boldsymbol{x}_5$:

In [ ]:
xd4 = NamedExpression(
    'xdotbm4',
    ((x4 - x3) / (t4 - t3) + (x5 - x4) / (t5 - t4)) / 2)
xd5 = NamedExpression(
    'xdotbm5',
    ((x5 - x4) / (t5 - t4) + (x6 - x5) / (t6 - t5)) / 2)
display(xd4, xd5)

To simplify the results,
we define a few symbols $\Delta_i = t_{i+1} - t_i$.
However, we are only using these for display purposes,
the calculations are still done with $t_i$.

In [ ]:
delta3, delta4, delta5 = sp.symbols('Delta3:6')
deltas = {
    t3: 0,
    t4: delta3,
    t5: delta3 + delta4,
    t6: delta3 + delta4 + delta5,
}

We are using some definitions from
[the notebook about non-uniform Hermite splines](hermite-non-uniform.ipynb),
namely the Hermite control values ...

In [ ]:
control_values_H = sp.Matrix(sp.symbols('xbm4:6 xdotbm4:6'))
control_values_H

... and the Hermite basis matrix:

In [ ]:
M_H = NamedMatrix(
    r'{M_{\text{H},4}}',
    sp.Matrix([[2, -2, 1, 1],
               [-3, 3, -2, -1],
               [0, 0, 1, 0],
               [1, 0, 0, 0]]) * sp.diag(1, 1, t5 - t4, t5 - t4))
M_H.subs(deltas)

Now we are looking for a matrix
that can transform our control points into Hermite control values.

In [ ]:
M_FDtoH = NamedMatrix(r'{M_{\text{FD},4\to\text{H},4}}', 4, 4)

In [ ]:
NamedMatrix(control_values_H, M_FDtoH.name * control_values_FD)

Left-multiplying this matrix with the Hermite basis matrix
will then result in the basis matrix for finite difference splines:

In [ ]:
M_FD = NamedMatrix(r'{M_{\text{FD},4}}', M_H.name * M_FDtoH.name)
M_FD

The matrix coefficients can be obtained from
the definition of the tangent vectors:

In [ ]:
M_FDtoH.expr = sp.Matrix([
    [expr.expand().coeff(cv) for cv in control_values_FD]
    for expr in control_values_H.subs([xd4.args, xd5.args])])
M_FDtoH.subs(deltas)

In case you want to go into the other direction
(from *Hermite* to *finite difference*),
you can invert the matrix:

In [ ]:
M_HtoFD = NamedMatrix(r'{M_{\text{H},4\to\text{FD},4}}', M_FDtoH.I.expr)
M_HtoFD.subs(deltas).expand()

To get the basis matrix,
we just have to do the matrix multiplication:

In [ ]:
M_FD = M_FD.subs_symbols(M_H, M_FDtoH).doit()
M_FD.subs(deltas).expand()

In [ ]:
M_FD.subs(deltas).I.expand()

Just as a quick check,
using unit parameter intervals
should lead to the *uniform* basis matrix:

In [ ]:
uniform = {
    t3: 3,
    t4: 4,
    t5: 5,
    t6: 6,
    M_FD.name: sp.Symbol(r'{M_\text{FD,uniform}}'),
}

In [ ]:
M_FD.subs(uniform).pull_out(sp.S.Half)

If everything went well,
this should be the same matrix
as the basis matrix for
[uniform Catmull--Rom splines](catmull-rom-uniform.ipynb).